In [5]:
import pandas as pd 
import gensim
from gensim.models.ldamodel import LdaModel

# Paths

In [15]:
video_data = "data/videos_parsed"
path_stopwords = "stopwords/serbo_croatian.csv"

# Load data

In [13]:
def load_stopwords(file_path):
    """ Loads stopword file, returns a list of words
    """
    df = pd.read_csv(file_path)
    stopwords = list(df.values.flatten())
    first_word = df.columns[0]
    stopwords.append(first_word)
    return stopwords

In [17]:
stopwords = load_stopwords(path_stopwords)
# stopwords

In [10]:
data = pd.read_pickle(video_data)
data.head()

,title,published,tags,comments,likes,dislikes,favourites,views,guest_name,quote,tokens,2_gram,3_gram,page_name,year,month,month_from_start,week_day
0,INTERVJU: Slobodan Komazec - Vučić uništava dr...,2018-03-15,"[Balkan Info Intervju, Balkan Info, Балкан Инф...",88,250,19,0,9432,Slobodan Komazec,"Vučić uništava državu, jer ne zna ništa o ekon...","[vučić, uništava, državu, jer, ne, zna, ništa,...","[vučić uništava, uništava državu, državu jer, ...","[vučić uništava državu, uništava državu jer, d...",Slobodan_Komazec,2018,3,34,Thursday
1,INTERVJU: Jelena Maćić - Opozicija će ovakvim ...,2018-03-13,"[Balkan Info Intervju, Balkan Info, Балкан Инф...",465,493,254,0,51341,Jelena Maćić,Opozicija će ovakvim ponašanjem biti skroz izb...,"[opozicija, će, ovakvim, ponašanjem, biti, skr...","[opozicija će, će ovakvim, ovakvim ponašanjem,...","[opozicija će ovakvim, će ovakvim ponašanjem, ...",Jelena_Maćić,2018,3,34,Tuesday
2,INTERVJU: Zoran Živković - Đinđića su ubili on...,2018-03-12,"[Balkan Info, Балкан Инфо, Теша Тешановић, Teš...",280,231,547,0,20028,Zoran Živković,Đinđića su ubili oni koji su se plašili modern...,"[đinđića, su, ubili, oni, koji, su, se, plašil...","[đinđića su, su ubili, ubili oni, oni koji, ko...","[đinđića su ubili, su ubili oni, ubili oni koj...",Zoran_Živković,2018,3,34,Monday
3,INTERVJU: Dragan Petrović - Bliži se poslednji...,2018-03-11,"[Balkan Info, Балкан Инфо, Теша Тешановић, Teš...",298,693,81,0,51857,Dragan Petrović,Bliži se poslednji trenutak da sprečimo izdaju...,"[bliži, se, poslednji, trenutak, da, sprečimo,...","[bliži se, se poslednji, poslednji trenutak, t...","[bliži se poslednji, se poslednji trenutak, po...",Dragan_Petrović,2018,3,34,Sunday
4,INTERVJU: Petar Čelik - Svetom vladaju oni koj...,2018-03-10,"[Balkan Info Intervju, Balkan Info, Балкан Инф...",240,727,74,0,49875,Petar Čelik,Svetom vladaju oni koji kontrolišu sve tokove ...,"[svetom, vladaju, oni, koji, kontrolišu, sve, ...","[svetom vladaju, vladaju oni, oni koji, koji k...","[svetom vladaju oni, vladaju oni koji, oni koj...",Petar_Čelik,2018,3,34,Saturday


# Removing Stopwords

In [22]:
def remove_stopwords(words, stopwords):
    """ Returns a lsit of 'informative' words
    """
    new_words = []
    for word in words:
        if word not in stopwords:
            new_words.append(word)
    return new_words

In [27]:
MISSING_STOPWORDS = ["i", "da", "je","u", "su", "na", "se", "za", "ne",
                     "će", "od", "sam", "iz", "samo", "sa", "mi", "o", "jer", "do",
                    "ali","ako","do","što","bili","bi","treba", "smo", "nama", "nam",
                     "a", "me", "to", "sebe", "svi", "deo", "nema", "čeka", "svih",
                    "neće", "našu", "iza", "ceo", "širom", "naši", "naša", "među", 
                    "celu","ga", "uz", "bila", "gori", "već", "niko", "sme", "pred", "mu",
                   "ću", "pa", "po", "bilo", "moji", "našeg", "bih", "ni", "ili", "tu", "ovu"]

In [28]:
stopwords = stopwords+MISSING_STOPWORDS

In [30]:
data["tokens_stopwords"] = data["tokens"].apply(lambda x: remove_stopwords(x, stopwords))
data.head()

,title,published,tags,comments,likes,dislikes,favourites,views,guest_name,quote,tokens,2_gram,3_gram,page_name,year,month,month_from_start,week_day,tokens_stopwords
0,INTERVJU: Slobodan Komazec - Vučić uništava dr...,2018-03-15,"[Balkan Info Intervju, Balkan Info, Балкан Инф...",88,250,19,0,9432,Slobodan Komazec,"Vučić uništava državu, jer ne zna ništa o ekon...","[vučić, uništava, državu, jer, ne, zna, ništa,...","[vučić uništava, uništava državu, državu jer, ...","[vučić uništava državu, uništava državu jer, d...",Slobodan_Komazec,2018,3,34,Thursday,"[vučić, uništava, državu, zna, ekonomiji]"
1,INTERVJU: Jelena Maćić - Opozicija će ovakvim ...,2018-03-13,"[Balkan Info Intervju, Balkan Info, Балкан Инф...",465,493,254,0,51341,Jelena Maćić,Opozicija će ovakvim ponašanjem biti skroz izb...,"[opozicija, će, ovakvim, ponašanjem, biti, skr...","[opozicija će, će ovakvim, ovakvim ponašanjem,...","[opozicija će ovakvim, će ovakvim ponašanjem, ...",Jelena_Maćić,2018,3,34,Tuesday,"[opozicija, ovakvim, ponašanjem, skroz, izbris..."
2,INTERVJU: Zoran Živković - Đinđića su ubili on...,2018-03-12,"[Balkan Info, Балкан Инфо, Теша Тешановић, Teš...",280,231,547,0,20028,Zoran Živković,Đinđića su ubili oni koji su se plašili modern...,"[đinđića, su, ubili, oni, koji, su, se, plašil...","[đinđića su, su ubili, ubili oni, oni koji, ko...","[đinđića su ubili, su ubili oni, ubili oni koj...",Zoran_Živković,2018,3,34,Monday,"[đinđića, ubili, plašili, moderne, srbije]"
3,INTERVJU: Dragan Petrović - Bliži se poslednji...,2018-03-11,"[Balkan Info, Балкан Инфо, Теша Тешановић, Teš...",298,693,81,0,51857,Dragan Petrović,Bliži se poslednji trenutak da sprečimo izdaju...,"[bliži, se, poslednji, trenutak, da, sprečimo,...","[bliži se, se poslednji, poslednji trenutak, t...","[bliži se poslednji, se poslednji trenutak, po...",Dragan_Petrović,2018,3,34,Sunday,"[bliži, poslednji, trenutak, sprečimo, izdaju,..."
4,INTERVJU: Petar Čelik - Svetom vladaju oni koj...,2018-03-10,"[Balkan Info Intervju, Balkan Info, Балкан Инф...",240,727,74,0,49875,Petar Čelik,Svetom vladaju oni koji kontrolišu sve tokove ...,"[svetom, vladaju, oni, koji, kontrolišu, sve, ...","[svetom vladaju, vladaju oni, oni koji, koji k...","[svetom vladaju oni, vladaju oni koji, oni koj...",Petar_Čelik,2018,3,34,Saturday,"[svetom, vladaju, kontrolišu, tokove, novca, s..."


# Dictionary

In [44]:
dictionary = gensim.corpora.Dictionary(data.tokens.values)

In [45]:
print(dictionary)

Dictionary(1768 unique tokens: ['državu', 'ekonomiji', 'jer', 'ne', 'ništa']...)


In [46]:
print(dictionary.token2id)

{'državu': 0, 'ekonomiji': 1, 'jer': 2, 'ne': 3, 'ništa': 4, 'o': 5, 'uništava': 6, 'vučić': 7, 'zna': 8, 'biti': 9, 'izbrisana': 10, 'mape': 11, 'opozicija': 12, 'ovakvim': 13, 'ponašanjem': 14, 'sa': 15, 'skroz': 16, 'će': 17, 'koji': 18, 'moderne': 19, 'oni': 20, 'plašili': 21, 'se': 22, 'srbije': 23, 'su': 24, 'ubili': 25, 'đinđića': 26, 'bliži': 27, 'da': 28, 'izdaju': 29, 'kosova': 30, 'poslednji': 31, 'sprečimo': 32, 'trenutak': 33, 'iz': 34, 'kontrolišu': 35, 'novca': 36, 'senke': 37, 'sve': 38, 'svetom': 39, 'tokove': 40, 'vladaju': 41, 'je': 42, 'mora': 43, 'na': 44, 'narod': 45, 'probudi': 46, 'sedativima': 47, 'sna': 48, 'srbija': 49, 'bili': 50, 'deo': 51, 'i': 52, 'naše': 53, 'odrastanja': 54, 'poroci': 55, 'rokenrola': 56, 'u': 57, 'vreme': 58, 'dokumenta': 59, 'istoriji': 60, 'naroda': 61, 'pravoj': 62, 'skriva': 63, 'srpskog': 64, 'vatikan': 65, 'amerike': 66, 'carstvo': 67, 'globalističko': 68, 'kraju': 69, 'svom': 70, 'više': 71, 'istorija': 72, 'milu': 73, 'njegove'

In [47]:
corpus = [dictionary.doc2bow(text) for text in data.tokens_stopwords.values]

In [48]:
corpus[:10]

[[(0, 1), (1, 1), (6, 1), (7, 1), (8, 1)],
 [(10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (16, 1)],
 [(19, 1), (21, 1), (23, 1), (25, 1), (26, 1)],
 [(27, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1)],
 [(35, 1), (36, 1), (37, 1), (39, 1), (40, 1), (41, 1)],
 [(45, 1), (46, 1), (47, 1), (48, 1), (49, 1)],
 [(53, 1), (54, 1), (55, 1), (56, 1), (58, 1)],
 [(59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1)],
 [(27, 1), (66, 1), (67, 1), (68, 1), (69, 1), (70, 1)],
 [(72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (78, 1)]]

In [49]:
lda = LdaModel(corpus, num_topics=10, id2word = dictionary, passes=50)

In [50]:
lda.show_topics(num_topics=10, num_words=10)

[(0,
  '0.014*"vučić" + 0.010*"srbiji" + 0.010*"kontrolišu" + 0.007*"ljudi" + 0.007*"mihailović" + 0.007*"draža" + 0.005*"rat" + 0.005*"uništavaju" + 0.005*"zato" + 0.005*"srpsku"'),
 (1,
  '0.009*"političari" + 0.009*"srbija" + 0.009*"uvek" + 0.009*"naše" + 0.007*"balkanu" + 0.007*"moramo" + 0.007*"godina" + 0.005*"vlast" + 0.005*"opozicija" + 0.005*"sukoba"'),
 (2,
  '0.017*"naroda" + 0.015*"srpskog" + 0.008*"vučiću" + 0.008*"bolesti" + 0.005*"srbi" + 0.005*"izdaje" + 0.005*"srbijom" + 0.005*"istoriji" + 0.005*"vladaju" + 0.005*"tajne"'),
 (3,
  '0.032*"vučić" + 0.012*"vučića" + 0.012*"srbiju" + 0.012*"kosova" + 0.010*"naroda" + 0.008*"žele" + 0.008*"narod" + 0.008*"izdaju" + 0.008*"srpski" + 0.006*"preti"'),
 (4,
  '0.013*"kosova" + 0.009*"tajne" + 0.009*"srbije" + 0.009*"službe" + 0.007*"vlasti" + 0.007*"žele" + 0.007*"srbija" + 0.007*"nećemo" + 0.007*"država" + 0.005*"smemo"'),
 (5,
  '0.028*"vučić" + 0.011*"imaju" + 0.011*"unište" + 0.008*"žele" + 0.008*"srbiju" + 0.008*"život" +

# Topic Modeling

In [51]:
def show_topics(token_lst, num_topics=5, num_words=10):
    """ Shows words (num+words) from different topics (num_topics) 
    """
    dictionary = gensim.corpora.Dictionary(token_lst)
    corpus = [dictionary.doc2bow(text) for text in token_lst]
    lda = LdaModel(corpus, num_topics=10, id2word = dictionary, passes=50)
    return lda.show_topics(num_topics=num_topics, num_words=num_words)

In [52]:
show_topics(data.tokens_stopwords.values)

[(7,
  '0.022*"vučić" + 0.018*"vučića" + 0.009*"političari" + 0.007*"nas" + 0.007*"dozvoliti" + 0.007*"vodi" + 0.005*"naroda" + 0.005*"moramo" + 0.005*"sistem" + 0.005*"država"'),
 (1,
  '0.024*"naroda" + 0.022*"vučić" + 0.021*"srpskog" + 0.017*"vlada" + 0.012*"narod" + 0.008*"srpski" + 0.008*"srbiji" + 0.008*"unište" + 0.007*"istoriju" + 0.006*"žele"'),
 (5,
  '0.018*"kosova" + 0.011*"izdaju" + 0.011*"vučić" + 0.009*"uvek" + 0.009*"metohije" + 0.007*"evropska" + 0.007*"srbija" + 0.007*"rat" + 0.007*"unija" + 0.005*"vreme"'),
 (4,
  '0.025*"vučić" + 0.014*"srbiju" + 0.011*"mafija" + 0.007*"zna" + 0.007*"srbija" + 0.007*"državu" + 0.007*"broz" + 0.007*"kraja" + 0.007*"naroda" + 0.007*"politička"'),
 (9,
  '0.017*"vučić" + 0.015*"narod" + 0.012*"vlasti" + 0.012*"srbi" + 0.010*"službe" + 0.010*"tajne" + 0.010*"srpski" + 0.008*"može" + 0.008*"želimo" + 0.008*"lažne"')]

In [53]:
show_topics(data.tokens.values)

[(7,
  '0.042*"vučić" + 0.035*"i" + 0.026*"je" + 0.014*"u" + 0.012*"koji" + 0.010*"će" + 0.010*"kosova" + 0.009*"su" + 0.009*"na" + 0.008*"srpski"'),
 (0,
  '0.029*"i" + 0.026*"je" + 0.024*"da" + 0.011*"na" + 0.010*"naroda" + 0.009*"evropska" + 0.008*"će" + 0.008*"mi" + 0.007*"od" + 0.007*"srpskog"'),
 (8,
  '0.046*"i" + 0.041*"da" + 0.031*"je" + 0.023*"u" + 0.020*"vučić" + 0.009*"su" + 0.008*"želi" + 0.007*"kosovo" + 0.007*"kosova" + 0.007*"se"'),
 (5,
  '0.029*"i" + 0.021*"da" + 0.014*"na" + 0.012*"u" + 0.012*"je" + 0.010*"narod" + 0.007*"sam" + 0.007*"srušiti" + 0.007*"ne" + 0.007*"se"'),
 (3,
  '0.036*"je" + 0.022*"i" + 0.020*"da" + 0.014*"su" + 0.014*"vučić" + 0.012*"na" + 0.008*"od" + 0.006*"uništavaju" + 0.006*"ali" + 0.006*"opasnost"')]